# Web Scrape

#### Step 1: Import libraries

#### Step 2: Create a list with all urls

#### Step 3: Loop through each link:
* Get html
* Build table
* Concat all tables

#### Step 4: Expand rows with compact values

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
filterDay = ['yesterday', 'today', 'tomorrow']

filterHour = ['0', '6', '12', '18']

urls = [f'https://www.santiago-airport.com/scl-departures?day={day}&tp={hour}' for day in filterDay for hour in filterHour]

In [3]:
dataframes = []

for url in urls:
    
    request = requests.get(url)    
    soup = BeautifulSoup(request.text, 'html.parser')

    Reference_Day = url.split('=')[1][:-3]
    if Reference_Day == 'Tomorrow':
        Date = soup.find('option',{'value':'?day=tomorrow'}).text.split(' ')[0]
    else:
        Date = soup.find('option',{'selected':"selected"}).text.split(' ')[0]

    flights = soup.find_all('div', class_='flight-row')
    rows = []

    for flight in flights[1:]:
        row={}
        if flight.find('div',{'class','adsense'}): # advertisment row
            continue
        row['Destination'] = flight.find('div', {'class','flight-col flight-col__dest-term'}).find('b').text
        row['Destination Code'] = flight.find('div', {'class','flight-col flight-col__dest-term'}).find('span').text
        row['Departure'] = flight.find('div', {'class','flight-col flight-col__hour'}).text.strip()
        row['Flight'] = flight.find('div', {'class','flight-col flight-col__flight'}).text.strip()
        row['Airline'] = flight.find('div', {'class','flight-col flight-col__airline'}).text.strip()
        row['Terminal'] = flight.find('div', {'class','flight-col flight-col__terminal'}).text.strip()
        row['Status'] = flight.find_all('a')[-1].text
        row['Date'] = Date
        row['Reference Day'] = Reference_Day
        row['url'] = url
        rows.append(row)

    data = pd.DataFrame(rows)
    dataframes.append(data)

In [4]:
df = pd.concat(dataframes)

In [7]:
# df_departures = pd.DataFrame(columns=['Destination','Destination Code','Departure','Flight','Airline','Terminal','Status','Date','Reference Day','url'])

# for index, row in df.iterrows():
#     values_b = row['Flight'].split('\n')
#     values_c = row['Airline'].split('\n')
#     n_values = max(len(values_b), len(values_c))
#     rows = []
#     for i in range(n_values):
#         new_row = {
#             'Destination': row['Destination'],
#             'Destination Code': row['Destination Code'],
#             'Departure': row['Departure'],
#             'Terminal': row['Terminal'],
#             'Status': row['Status'],
#             'Date': row['Date'],
#             'Reference Day': row['Reference Day'],
#             'url': row['url']
#             }
#         if i < len(values_b):
#             new_row['Flight'] = values_b[i]
#         else:
#             new_row['Flight'] = ''
#         if i < len(values_c):
#             new_row['Airline'] = values_c[i]
#         else:
#             new_row['Airline'] = ''
#         rows.append(new_row)
#     df_departures = pd.concat([df_departures, pd.DataFrame(rows)], ignore_index=True)

df_departures = pd.DataFrame(columns=['Destination', 'Destination Code', 'Departure', 'Flight', 'Airline', 'Terminal', 'Status', 'Date', 'Reference Day', 'url'])

for index, row in df.iterrows():
    values_b = row['Flight'].split('\n')
    values_c = row['Airline'].split('\n')
    n_values = max(len(values_b), len(values_c))
    rows = []
    for i in range(n_values):
        # use a dictionary comprehension to create a new row for each value
        new_row = {
            key: row[key]
            if key not in ['Flight', 'Airline']
            else (
                values_b[i]
                if key == 'Flight' and i < len(values_b)
                else values_c[i]
                    if key == 'Airline' and i < len(values_c)
                    else '')
            for key in df_departures.columns}
        rows.append(new_row)
    df_departures = pd.concat([df_departures, pd.DataFrame(rows)], ignore_index=True)


In [8]:
df_departures

,Destination,Destination Code,Departure,Flight,Airline,Terminal,Status,Date,Reference Day,url
0,Panama City,(PTY),00:26,CM118,Copa Airlines,2,Landed - On-time [+],2023-04-24,yesterday,https://www.santiago-airport.com/scl-departure...
1,Bogota,(BOG),00:30,LA578,LATAM Airlines,2,Landed - On-time [+],2023-04-24,yesterday,https://www.santiago-airport.com/scl-departure...
2,Bogota,(BOG),00:30,DL7387,Delta Air Lines,2,Landed - On-time [+],2023-04-24,yesterday,https://www.santiago-airport.com/scl-departure...
3,Bogota,(BOG),00:30,QF4207,Qantas,2,Landed - On-time [+],2023-04-24,yesterday,https://www.santiago-airport.com/scl-departure...
4,Auckland,(AKL),00:35,LA801,LATAM Airlines,2,Landed - Delayed [+],2023-04-24,yesterday,https://www.santiago-airport.com/scl-departure...
...,...,...,...,...,...,...,...,...,...,...
776,New York,(JFK),22:55,LA532,LATAM Airlines,2,Scheduled [+],2023-04-26,tomorrow,https://www.santiago-airport.com/scl-departure...
777,New York,(JFK),22:55,DL6061,Delta Air Lines,2,Scheduled [+],2023-04-26,tomorrow,https://www.santiago-airport.com/scl-departure...
778,New York,(JFK),22:55,JL7627,JAL Japan Airlines,2,Scheduled [+],2023-04-26,tomorrow,https://www.santiago-airport.com/scl-departure...
779,Miami,(MIA),23:20,LA500,LATAM Airlines,2,Scheduled [+],2023-04-26,tomorrow,https://www.santiago-airport.com/scl-departure...
